In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import requests
#import ydata profiling
from ydata_profiling import ProfileReport
import numpy as np
import re

In [ ]:
# import religion and election excel file:

df_rv = pd.read_excel('ReligionData.xlsx',sheet_name='Religion and Election Data')
pd.set_option('display.max_columns', None)
df_rv = df_rv.fillna(0)

# remove columns:

df_rv = df_rv[df_rv.columns.drop(list(df_rv.filter(regex='per Thousand')))]
df_rv = df_rv[df_rv.columns.drop(list(df_rv.filter(regex='Per Thousand')))]
df_rv = df_rv[df_rv.columns.drop(list(df_rv.filter(regex='Buddhism')))]
df_rv = df_rv.drop([
'Indian-American Hindu Adherents',
'Post Renaissance Hindu Adherents',
'Renaissance Hindu Adherents',
'Traditional Hindu Temples Adherents',
'Conservative Judaism Adherents',
'Orthodox Judaism Adherents',
'Reconstructionist Judaism Adherents',
'Reform Judaism Adherents'], axis=1)

# create state mapping dictionary:

us_state_abbrev = {
    'AL': 'Alabama',
    'AK': 'Alaska',
    'AZ': 'Arizona',
    'AR': 'Arkansas',
    'CA': 'California',
    'CO': 'Colorado',
    'CT': 'Connecticut',
    'DC': 'District of Columbia',
    'DE': 'Delaware',
    'FL': 'Florida',
    'GA': 'Georgia',
    'HI': 'Hawaii',
    'ID': 'Idaho',
    'IL': 'Illinois',
    'IN': 'Indiana',
    'IA': 'Iowa',
    'KS': 'Kansas',
    'KY': 'Kentucky',
    'LA': 'Louisiana',
    'ME': 'Maine',
    'MD': 'Maryland',
    'MA': 'Massachusetts',
    'MI': 'Michigan',
    'MN': 'Minnesota',
    'MS': 'Mississippi',
    'MO': 'Missouri',
    'MT': 'Montana',
    'NE': 'Nebraska',
    'NV': 'Nevada',
    'NH': 'New Hampshire',
    'NJ': 'New Jersey',
    'NM': 'New Mexico',
    'NY': 'New York',
    'NC': 'North Carolina',
    'ND': 'North Dakota',
    'OH': 'Ohio',
    'OK': 'Oklahoma',
    'OR': 'Oregon',
    'PA': 'Pennsylvania',
    'RI': 'Rhode Island',
    'SC': 'South Carolina',
    'SD': 'South Dakota',
    'TN': 'Tennessee',
    'TX': 'Texas',
    'UT': 'Utah',
    'VT': 'Vermont',
    'VA': 'Virginia',
    'WA': 'Washington',
    'WV': 'West Virginia',
    'WI': 'Wisconsin',
    'WY': 'Wyoming',
}

In [ ]:
# make updates to df_rv dataframe:

df_rv['State Code'] = df_rv.replace({'State Code':us_state_abbrev})  # update State Code with full state name
df_rv['County Name'] = df_rv['County Name'].map(str) + ', ' + df_rv['State Code'].map(str) # concatenate county and state

# rename df_rv columns:

df_rv.rename(columns={'Clinton or Trump State':'Party Affiliation',
                     'Clinton':'Democrat','Trump':'Republican',
                     '% Clinton':'% Democrat','% Trump':'% Republican',
                     'County Name':'Geographic Area Name',
                     'State Code':'State_Code'}, inplace=True)

# update Party Affiliation:

df_rv['Party Affiliation'] = df_rv['Party Affiliation'].replace({'Trump':'Republican','Clinton':'Democrat'})


In [ ]:
https://youtu.be/Px3_lDaXHJM    # df_rv.head()
# print(df_rv.info())

# print(df_dem.info()) # Geographic Area Name 838 non-null object
# df_dem['Geographic Area Name'].unique

df_rv['Geographic Area Name'].unique

In [ ]:
print(df_rv.dtypes)
    
chk = df_rv.select_dtypes(include ='object')
chk.columns

In [ ]:
df_rv.info()

In [ ]:
df_rv[df_rv['State_Code']=='Alabama']
# df_rv['Party Affiliation'].unique()

In [ ]:
# import demographic and housing estimates csv:

df_dem = pd.read_csv('2018 ACS DP05 Demographic and Housing Estimates.csv',header=1)
df_dem = df_dem[df_dem.columns.drop(list(df_dem.filter(regex='Margin of Error!!')))]  # drop columns with MoE
df_dem = df_dem[df_dem.columns.drop(list(df_dem.filter(regex='Percent Estimate!!')))] # drop columns with PctEst
pd.set_option('display.max_columns', None)
df_dem = df_dem.fillna(0) # update nan columns with zero


# replace invalid values:

c = df_dem.columns[2:]
for x in c:
    df_dem[x].replace('N',0,inplace=True)
    df_dem[x].replace('(X)',0,inplace=True)
    df_dem[x].replace('None',0,inplace=True)

# change data types:

c = df_dem.columns[35:]
for x in c:
    df_dem[x] = df_dem[x].astype(int)

# df_dem.apply(pd.to_numeric, errors='ignore')
    
df_dem.head()

In [ ]:
# check to see if there are any df_dem objects:

print(df_dem.dtypes)
    
chk1 = df_dem.select_dtypes(include ='object')
chk1.columns

In [ ]:
# import housing characteristics csv:

df_hou = pd.read_csv('2018 ACS DP04 Housing Characteristics.csv',header=1)
df_hou = df_hou[df_hou.columns.drop(list(df_hou.filter(regex='Margin of Error!!')))]
df_hou = df_hou[df_hou.columns.drop(list(df_hou.filter(regex='Percent Estimate!!')))]
pd.set_option('display.max_columns', None)
df_hou = df_hou.fillna(0)

# replace invalid values:

c = df_hou.columns[2:]
for x in c:
    df_hou[x].replace('N',0,inplace=True)
    df_hou[x].replace('(X)',0,inplace=True)
    df_hou[x].replace('None',0,inplace=True)
    
    
df_hou['Estimate!!HOUSING OCCUPANCY!!Total housing units!!Homeowner vacancy rate'] = df_hou['Estimate!!HOUSING OCCUPANCY!!Total housing units!!Homeowner vacancy rate'].astype(float)

df_hou['Estimate!!HOUSING OCCUPANCY!!Total housing units!!Rental vacancy rate'] = df_hou['Estimate!!HOUSING OCCUPANCY!!Total housing units!!Rental vacancy rate'].astype(float)

# change data types for df_hou objects:

c1 = df_hou.select_dtypes(include ='object')
c1.columns[2:]


# change data types to int
c = c1.columns[2:]
for x in c:
    df_hou[x] = df_hou[x].astype(int)
    
df_hou.head()

In [ ]:
# check to see if data type modification took place:

print(df_hou.dtypes)

chk2 = df_hou.select_dtypes(include ='object')
chk2.columns

In [ ]:
pd.set_option('display.max_rows', None)
df_hou.dtypes

In [ ]:
# c1.columns[4:]
# c1.columns[2:4]
# df_hou.select_dtypes(include ='object')

In [ ]:
# import economic charateristics csv:

df_econ = pd.read_csv('2018 ACS DP03 Economic Characteristics.csv',header=1)
df_econ = df_econ[df_econ.columns.drop(list(df_econ.filter(regex='Margin of Error!!')))]
df_econ = df_econ[df_econ.columns.drop(list(df_econ.filter(regex='Percent Estimate!!')))]
pd.set_option('display.max_columns', None)
df_econ = df_econ.fillna(0)

# replace invalid values:

c = df_econ.columns[2:]
for x in c:
    df_econ[x].replace('N',0,inplace=True)
    df_econ[x].replace('(X)',0,inplace=True)
    df_econ[x].replace('None',0,inplace=True)
    
    
# change data types for df_hou objects:

c1 = df_econ.select_dtypes(include ='object')
c1.columns[2:]


# # change data types to float
# c = c1.columns[2:4]
# for x in c:
#     df_econ[x] = df_econ[x].astype(float)

    
# change data types to int
c = c1.columns[2:]
for x in c:
    df_econ[x] = df_econ[x].astype(int)
    
df_econ.head()

In [ ]:
chk3 = df_econ.select_dtypes(include ='object')
chk3.columns

In [ ]:
# import social characteristics csv:

df_soc = pd.read_csv('2018 ACS DP02 Social Characteristics.csv',header=1)
df_soc = df_soc[df_soc.columns.drop(list(df_soc.filter(regex='Margin of Error!!')))]
df_soc = df_soc[df_soc.columns.drop(list(df_soc.filter(regex='Percent Estimate!!')))]
pd.set_option('display.max_columns', None)
df_soc = df_soc.fillna(0)

# replace invalid values:

c = df_soc.columns[2:]
for x in c:
    df_soc[x].replace('N',0,inplace=True)
    df_soc[x].replace('(X)',0,inplace=True)
    df_soc[x].replace('None',0,inplace=True)
    
c1 = df_soc.select_dtypes(include ='object')
c1.columns[2:]


# # change data types to float
# c = c1.columns[2:4]
# for x in c:
#     df_econ[x] = df_econ[x].astype(float)

    
# change data types to int
c = c1.columns[2:]
for x in c:
    df_soc[x] = df_soc[x].astype(int)
    
df_soc.head()

In [ ]:
df_soc.dtypes
chk4 = df_soc.select_dtypes(include ='object')
chk4.columns

In [ ]:
# scrape legalization status by state:

url = "https://disa.com/map-of-marijuana-legality-by-state"
html = urlopen(url)
soup = BeautifulSoup(html)

# use getText()to extract the text we need into a list:
headers = [th.getText() for th in soup.findAll('tr', limit=2)[0].findAll('th')]
    
    
# avoid the first header row:
rows = soup.findAll('tr')[1:]

r = [[td.getText() for td in rows[i].findAll('td')]
            for i in range(len(rows))]


state_laws = pd.DataFrame(r, columns = headers)
state_laws = state_laws.drop(columns='State Laws')


# remove asterisks from multiple column values:

for x in state_laws.columns[1:]:
    state_laws[x].replace(regex=["\*"],value='',inplace=True)


state_laws.rename(columns={'State':'State_Code'}, inplace=True)
state_laws.head(100)

In [ ]:
# import PRRI excel file:

df_prri = pd.read_excel('PRRI_Religion.xlsx',sheet_name='Sheet 1',header=2)
pd.set_option('display.max_columns', None)
df_prri = df_prri.fillna(0)
df_prri = df_prri.drop(columns='Unnamed: 0')

In [ ]:
# check df_prri:

df_prri.head()

In [ ]:
# create merged dataframe:

df_merge = pd.merge(df_dem, df_hou, on=['id', 'Geographic Area Name'])
df_merge = pd.merge(df_merge, df_econ, on=['id', 'Geographic Area Name'])
df_merge = pd.merge(df_merge, df_soc, on=['id', 'Geographic Area Name'])

In [ ]:
pd.set_option('display.max_columns', None)
df_merge.info()
print(len(df_merge['Geographic Area Name']))
# df_merge['Geographic Area Name'].info()

In [ ]:
# assign state to dataframe:

w = []
for x in df_merge['Geographic Area Name']:
#     w.append(re.split(r',',x))
    splitting = x.split(',')
    last = splitting[1]
    l = last.strip()
    w.append(l)

df_merge['State_Code'] = w

# replace name of NM county:

df_merge['Geographic Area Name'].replace('Doña Ana County, New Mexico','Dona Ana County, New Mexico',inplace=True)


In [ ]:
# check df_merge:
pd.set_option('display.max_rows', 10)
df_merge.dtypes

In [ ]:
df1 = pd.merge(df_merge, df_rv, on= ['Geographic Area Name'],how='left')
df1.drop(columns=(['State_Code_y']),inplace = True)
df1.rename(columns={'State_Code_x':'State_Code'}, inplace=True)

c = df1.columns[1:]
for x in c:
    df1[x].replace('!','',inplace=True)
    df1[x].replace('"','',inplace=True)

df1.head()

In [ ]:
print(df_merge.dtypes)
    
chk5 = df_merge.select_dtypes(include ='object')
chk5.columns

In [ ]:
df = pd.merge(df1, state_laws, on= ['State_Code'])
df.head()

# print(state_laws.dtypes)
# df1.dtypes

# print(df1['State_Code'].unique())
# print(state_laws['State_Code'].unique())

# df.drop(columns=(['State Code_y']),inplace = True)
# df.rename(columns={'State Code_x':'State Code'}, inplace=True)

In [ ]:
display(
df_dem.info())
# [838 rows x 91 columns]>

df_hou.info()
# [838 rows x 145 columns]>

display(
df_econ.info())
# [838 rows x 139 columns]>

df_soc.info()
# [838 rows x 154 columns]>

display(
df_merge.info())
# [838 rows x 523 columns]>

df_rv.info()
# [3113 rows x 55 columns]>


display(df.info())
# Int64Index: 823 entries, 0 to 822

In [ ]:
unique = df_merge['Geographic Area Name'].unique()
for x in unique:
    print(x)

In [ ]:
unique = df['Geographic Area Name'].unique()
for x in unique:
    print(x)
    
# df.columns

In [ ]:
# replace column characters:

df.columns= df.columns.str.replace('!','',regex=True)  # replace exclamation marks
df.columns= df.columns.str.replace('$','',regex=True)  # replace dollar signs
df.columns= df.columns.str.replace("'",'',regex=True)  # replace apostrophes
df.columns= df.columns.str.replace(',','',regex=True)  # replace commas
df.columns= df.columns.str.replace('.','',regex=True)  # replace periods
df.columns= df.columns.str.replace('"','',regex=True)  # replace quotes 
df.columns = df.columns.str.replace('-', '_',regex=True)  # replace dashes with underscores
df.columns = df.columns.str.replace('(', '',regex=True)  # replace open parentheses
df.columns = df.columns.str.replace(')', '',regex=True)  # replace close parentheses
df.columns = df.columns.str.replace('%', 'Pct',regex=True)  # replace percentage sign
df.columns = df.columns.str.replace(' ', '',regex=True)  # replace white space in columns

# replace null values in dataframe:

df.fillna(0,inplace=True)

# update Alaska Party Affiliation:

df['PartyAffiliation'].replace(regex=[0],value='Republican',inplace=True)

# replace None values in dataframe:

c = df.columns[1:]
for x in c:
    df[x].replace('None',0,inplace=True)

for x in c:
    print(x)

In [ ]:
# use function to check for missing values:

def missing_values_table(df):
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        return mis_val_table_ren_columns
    
missing_values_table(df)

In [ ]:
# execute pandas profiling on df dataframe:

profile = df.profile_report()
rejected_variables = profile.get_rejected_variables(threshold=0.9)

In [ ]:
rejected_variables

In [ ]:
# inspect the objects for columns needed:

chk6 = df.select_dtypes(include ='object')
chk6.columns

In [ ]:
# drop unnecessary categories:

p = ['id','ANSICode', 'CBSATitle', 'Metropolitan/MicropolitanStatisticalArea',
       'CSATitle', 'MetropolitanDivisionTitle', 'Central/OutlyingCounty']

for x in p:
    df.drop([x],axis=1,inplace=True)

In [ ]:
# drop rejected variables from pandas profiling:

for x in rejected_variables:
    df.drop([x],axis=1,inplace=True)

In [ ]:
# execute pandas profiling again:

profile = df.profile_report()
rejected_variables = profile.get_rejected_variables(threshold=0.9)

In [ ]:
profile

In [ ]:
# check Party Affiliation for zeros:

df[df['PartyAffiliation']==0]

In [ ]:
# df.Zoroastrian_Adherents.unique()
c = df.columns[1:]
for x in c:
    df[x].replace('None',0,inplace=True)
#     df[x].fillna(value=np.nan, inplace=True)

In [ ]:
pd.set_option('display.max_rows', None)
# df_rv['County Name'].unique
df_dem['Geographic Area Name'].unique

In [ ]:
df.info()
c = df.columns[0:]

for x in c:
    print(x)
    
df.head()

In [ ]:
from pivottablejs import pivot_ui

import plotly.offline as py
py.init_notebook_mode(connected=True)
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.figure_factory as ff

import matplotlib.pyplot as plt 
%matplotlib inline
plt.style.use('ggplot')

import seaborn as sns 
sns.set_style('darkgrid')

In [ ]:
# check legalized distribution:

user_rating = df['Legalized'].value_counts()
plt.figure(figsize=(15,10))
plt.xlabel("Legal Status")
plt.ylabel("Count")
plt.title("Legalized Distribution")
sns.barplot(user_rating.index, user_rating.values, palette='bright');

In [ ]:
# check legal status distribution:

user_rating = df['LegalStatus'].value_counts()
plt.figure(figsize=(15,10))
plt.xlabel("Legal Status")
plt.ylabel("Count")
plt.title("Legal Status Distribution")
sns.barplot(user_rating.index, user_rating.values, palette='bright');

In [ ]:
print(df['LegalStatus'].value_counts())
df_copy['LegalStatus'].value_counts()
# df_copy[df_copy['LegalStatus']=='Fully Legal']

In [ ]:
# check party affiliation distribution:

user_rating = df['PartyAffiliation'].value_counts()
plt.figure(figsize=(15,10))
plt.xlabel("Party Affiliation")
plt.ylabel("Count")
plt.title("Political Party Distribution")
sns.barplot(user_rating.index, user_rating.values, palette='bright');

In [ ]:
# check medicinal distribution:

user_rating = df['Medicinal'].value_counts()
plt.figure(figsize=(15,10))
plt.xlabel("Medicinal Values")
plt.ylabel("Count")
plt.title("Medicinal Distribution")
sns.barplot(user_rating.index, user_rating.values, palette='bright');

In [ ]:
# check decriminalized distribution:

user_rating = df['Decriminalized'].value_counts()
plt.figure(figsize=(15,10))
plt.xlabel("Decriminalized Values")
plt.ylabel("Count")
plt.title("Decriminalized Distribution")
sns.barplot(user_rating.index, user_rating.values, palette='bright');

In [ ]:
sc = df.select_dtypes(include ='object')
sc.columns

In [ ]:
# make copy of dataframe:
# df_copy= df.copy()

df_copy.head()

In [ ]:
# cc = df.select_dtypes(include ='object')
# cc.columns
# 'GeographicAreaName',
# df.drop(['GeographicAreaName'],axis=1, inplace=True)

cc = ['GeographicAreaName','PartyAffiliation','Medicinal','Decriminalized']

for x in cc:
    df = pd.concat([df,pd.get_dummies(df[x], prefix=x)],axis=1)
    df.drop([x],axis=1, inplace=True)

# fix column headers for new columns:

df.columns.columns= df.columns.str.replace("'",'',regex=True)
df.columns= df.columns.str.replace(', ','_',regex=True)
df.columns= df.columns.str.replace(' ','_',regex=True)
df.columns= df.columns.str.replace('.','',regex=True)
    
# create features and target dataframes:

df_features = df.drop(['LegalStatus','State_Code'], axis=1)
df_target = pd.DataFrame(df['LegalStatus'])

df_features = df_features.astype(float)

In [ ]:
df_features = df_features.astype(float)

In [ ]:
# df_features.head()
# df_target.head()

for x in df_features.columns:
    print(df_features[x].dtypes)

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
from datetime import datetime

import itertools
from itertools import combinations, cycle

import statsmodels as sm
import statsmodels.api as sm
from statsmodels.formula.api import ols

import xgboost as xgb
from xgboost import plot_importance
from matplotlib import pyplot

from sklearn import svm
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
import sklearn.preprocessing as preprocessing
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, LabelEncoder, MultiLabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge, LassoCV, LassoLarsCV, LassoLarsIC, lasso_path, enet_path
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import confusion_matrix, classification_report, multilabel_confusion_matrix
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score
from sklearn.feature_selection import VarianceThreshold, f_regression, mutual_info_regression, SelectKBest, RFE, RFECV
from sklearn import datasets
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import log_loss
from sklearn.ensemble import VotingClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from scipy import stats
import scipy.stats as st
from scipy.stats import shapiro

from imblearn.over_sampling import SMOTE, ADASYN

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def prediction_function(model,train_x,test_x, train_y,test_y, estimate=None):
    
    cols = [i for i in df.columns if i not in df['Legalized']]
    
    model.fit(train_x,train_y)
    predictions = model.predict(test_x)
    probabilities = model.predict_proba(test_x)
    
    predict_train = model.predict(train_x)
    prob_train = model.predict_proba(train_x)
    
    
    if estimate == 'dec_tree':
        coefficients = pd.DataFrame(model.feature_importances_)
    elif estimate == 'rfe':
        coefficients = pd.DataFrame(model.estimator_.coef_.ravel())
    else:
        coefficients = pd.DataFrame(model.coef_.ravel())
    
    column_df = pd.DataFrame(cols)
    coef_sumry = (pd.merge(coefficients,column_df,left_index= True,right_index= True, how = "left"))
    coef_sumry.columns = ["coefficients","features"]
    coef_sumry = coef_sumry.sort_values(by = "coefficients",ascending = False)
    
    print (model)
    print ("\n Score Report : \n",classification_report(test_y,predictions))
    print ("Accuracy Score : ",accuracy_score(test_y,predictions))
    print ('Training Root Mean Square Error',np.sqrt(metrics.mean_squared_error(train_y,predict_train)))
    print ('Testing Root Mean Square Error',np.sqrt(metrics.mean_squared_error(test_y,predictions)))
    
    
    c_matrix = confusion_matrix(test_y,predictions)
    cnf_matrix = pd.DataFrame(c_matrix)
    
    model_roc_auc = roc_auc_score(test_y,predictions) 
    print ("AUC : ",model_roc_auc,"\n")
    test_fpr,test_tpr,test_thresholds = roc_curve(test_y, probabilities[:,1])
    train_fpr,train_tpr,train_thresholds = roc_curve(train_y, prob_train[:,1])
    
    trace1 = go.Heatmap(z = cnf_matrix ,
                        x = [0,1],
                        y = [0,1],
                        showscale  = False,colorscale = "Viridis",
                        name = "matrix")
    
    trace2 = go.Scatter(x = test_fpr,y = test_tpr,
                        name = "ROC Test",
                        line = dict(color = ('rgb(22, 96, 167)'),width = 2))
    
    trace3 = go.Scatter(x = [0,1],y=[0,1],
                        line = dict(color = ('rgb(205, 12, 24)'),width = 2,
                        dash = 'dot'))
    
    trace4 = go.Scatter(x = train_fpr,y=train_tpr,
                        name= "ROC Train",
                        line = dict(color = ('rgb(255, 128, 0)'),width = 2))
    
    trace5 = go.Bar(x = coef_sumry["features"],y = coef_sumry["coefficients"],
                    name = "coefficients",
                    marker = dict(color = coef_sumry["coefficients"],
                                  colorscale = "Picnic",
                                  line = dict(width = .6,color = "black")))
    
    fig = make_subplots(rows=2, cols=2, specs=[[{}, {}], [{'colspan': 2}, None]],
                            subplot_titles=('ROC Curve','Confusion Matrix','Feature Ranking'))
    
    fig.append_trace(trace1,1,2)
    fig.append_trace(trace2,1,1)
    fig.append_trace(trace3,1,1)
    fig.append_trace(trace4,1,1)
    fig.append_trace(trace5,2,1)
    
    fig['layout'].update(showlegend=False, title="Model Performance" ,
                         autosize = False,height = 900,width = 800,
                         plot_bgcolor = 'rgba(240,240,240, 0.95)',
                         paper_bgcolor = 'rgba(240,240,240, 0.95)',
                         margin = dict(b = 195))
    
    py.iplot(fig)

In [ ]:
def model_score(model,train_x,test_x,train_y,test_y,name):
    model.fit(train_x,train_y)
    predictions  = model.predict(test_x)
    accuracy     = accuracy_score(test_y,predictions)
    recallscore  = recall_score(test_y,predictions)
    precision    = precision_score(test_y,predictions)
    roc_auc      = roc_auc_score(test_y,predictions)
    f1score      = f1_score(test_y,predictions) 
    
    df = pd.DataFrame({"Model"           : [name],
                       "Accuracy_score"  : [accuracy],
                       "Recall_score"    : [recallscore],
                       "Precision"       : [precision],
                       "f1_score"        : [f1score],
                       "Area_under_curve": [roc_auc]})
    return df


In [ ]:
def model_compare(metric,color) :
    chart = go.Bar(y = model_performances["Model"] ,
                    x = model_performances[metric],
                    orientation = "h",name = metric ,
                    marker = dict(line = dict(width =.7),
                                  color = color))
    return chart

In [ ]:
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true, preds)
    f1 = f1_score(true, preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {}".format(acc))
    print("F1-Score: {}".format(f1))

In [ ]:
class MultiColumnLabelEncoder:
    
    def __init__(self, columns = None):
        self.columns = columns # list of column to encode
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        
        output = X.copy()
        
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname, col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        
        return output
    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)

In [ ]:
# use target encoding for categorical features:

def calc_smooth_mean(df, by, on, m):
    # Compute the global mean
    mean = df[on].mean()

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return df[by].map(smooth)


## Intermediate Models

### Conduct Test of Legalized Field

In [ ]:
# instantiate encoders:
le = LabelEncoder()
ohc = OneHotEncoder()

# make copy of dataframe:
df = df_copy.copy()

# ============================================================================== #

# select columns to be removed:
dcol = [
'GeographicAreaName',
'State_Code',
'EstimateRACETotalpopulationTwoormoreracesWhiteandAmericanIndianandAlaskaNative',
'EstimateRACETotalpopulationTwoormoreracesBlackorAfricanAmericanandAmericanIndianandAlaskaNative',
'EstimateRacealoneorincombinationwithoneormoreotherracesTotalpopulationAmericanIndianandAlaskaNative',
'VoteDifferenceC_T',
'VoteDifferenceT_C',
'2012TotalVotes',
'ClintonBWObama',
'TrumpBWRomney',
'ClintonPctBWObama',
'TrumpPctBWRomney',
'2010LandArea',
'Density',
'CombinedFIPSCode',
'CountyFIPSCode',
'CBSACode',
'CSACode',
'MetropolitanDivisionCode']


for x in dcol:
    df.drop([x], axis=1, inplace=True)

# ============================================================================== #

# remove columns where constant value equals zero:
df = df.loc[:, (df != df.iloc[0]).any()] 

# ============================================================================== #

# create column for testing:
df['Legalized']=df['LegalStatus']

# update test column:
df.iloc[:,-1].replace('Fully Legal','Yes',inplace=True)
df.iloc[:,-1].replace('Fully Illegal','No',inplace=True)
df.iloc[:,-1].replace('Mixed','No',inplace=True)

# drop columns:
df.drop(['LegalStatus'], axis=1,inplace=True)
df.drop(['Decriminalized'], axis=1, inplace=True)

# label encoding:
df['PartyAffiliation']= le.fit_transform(df['PartyAffiliation'])
df['Legalized']= le.fit_transform(df['Legalized'])

# combine adherents:
df1['Mathematics_score']=df1['Mathematics1_score'] + df1['Mathematics2_score']
print(df1)



In [ ]:
print('LegalStatus = ',df_copy['LegalStatus'].unique())
print('Decriminalized = ',df_copy['Decriminalized'].unique())
print('Medicinal = ',df_copy['Medicinal'].unique())
print('PartyAffiliation = ',df_copy['PartyAffiliation'].unique())
print('Legalized = ',df['Legalized'].unique())

# LegalStatus =  ['Fully Illegal' 'Fully Legal' 'Mixed'];  Fully Illegal = 0, Mixed = 1, Fully Legal = 2
# Decriminalized =  ['No' 'Yes' 'Reduced']; No = 0, Reduced = 1, Yes = 2
# Medicinal =  ['No' 'Yes' 'CBD Oil']; No = 0, CBD Oil = 1, Yes = 2
# PartyAffiliation =  ['Republican' 'Democrat']; Label Encoder
# Legalized =  ['No' 'Yes']; Label Encoder

In [ ]:
# label encoding:

# df['PartyAffiliation']= le.fit_transform(df['PartyAffiliation'])
# df['Legalized']= le.fit_transform(df['Legalized'])

In [ ]:
#target encoding:

# df['GeographicAreaName'] = calc_smooth_mean(df, by='GeographicAreaName', on='Legalized', m=10)

In [ ]:
# ordinal encoding:

# ordinalencoder = OrdinalEncoder()
# ordinalencoder.fit_transform(df[['LegalStatus']])

# # Using pandas factorize method for ordinal data
# categories1 = pd.Categorical(df['LegalStatus'], categories=['Fully Illegal', 'Mixed', 'Fully Legal'], ordered=True) # Order of labels set for data


# # Factorizing the column data
# labels1, unique = pd.factorize(categories1, sort=True)
# df['LegalStatus'] = labels1  # Encoded Legal Status


# ordinalencoder.fit_transform(df[['Decriminalized']])

# # Using pandas factorize method for ordinal data
# categories2 = pd.Categorical(df['Decriminalized'], categories=['No', 'Reduced', 'Yes'], ordered=True) # Order of labels set for data

# # Factorizing the column data
# labels2, unique = pd.factorize(categories2, sort=True)
# df['Decriminalized'] = labels2  # Encoded Legal Status


# ordinalencoder.fit_transform(df[['Medicinal']])

# # Using pandas factorize method for ordinal data
# categories3 = pd.Categorical(df['Medicinal'], categories=['No', 'CBD Oil', 'Yes'], ordered=True) # Order of labels set for data

# # Factorizing the column data
# labels3, unique = pd.factorize(categories3, sort=True)
# df['Medicinal'] = labels3  # Encoded Legal Status



In [ ]:
# binary encoding for all 50 states:

# import category_encoders as ce

# encoder = ce.BinaryEncoder(cols=['State_Code'])
# df = encoder.fit_transform(df)


In [ ]:
#target encoding:
chk = df.select_dtypes(include ='object')
c = chk.columns

for x in chk.columns:
    df[x] = calc_smooth_mean(df, by=x, on='Legalized', m=10)

In [ ]:
# check categorical columns:
chk = df.select_dtypes(include ='object')
chk.columns

In [ ]:
# check for columns with zeros comprising at least 50%:
cols = list(df.columns[df[df == 0].count(axis=0)/len(df.index)>0.5])
cols

In [ ]:
# find columns where zero count > 50%:
cols = list(df.columns[df[df == 0].count(axis=0)/len(df.index)>0.5])

# remove items that should not be dropped from dataframe:
# for x in ['State_Code_0','State_Code_1', 'State_Code_2', 'State_Code_3', 'State_Code_4','State_Code_5', 'State_Code_6',
# 'Legalized']:
#     cols.remove(x)

for x in ['Legalized']:
    cols.remove(x)
    
# drop columns from dataframe:
for x in cols:
    df.drop([x], axis=1,inplace=True)


In [ ]:
df.head()

In [ ]:
# create features and target dataframes:

df_features = df.drop(['Legalized'], axis=1)
df_target = df['Legalized']

In [ ]:
profile_new = df.profile_report()
rejected_variables_new = profile_new.get_rejected_variables(threshold=0.9)


In [ ]:
rejected_variables_new

In [ ]:
profile_new

### Train/Test Split

In [ ]:
X = df_features
y = df_target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

logreg = LogisticRegression() 
baseline_log = logreg.fit(X_train, y_train)
print(baseline_log) 

In [ ]:
prediction_function(baseline_log, X_train, X_test, y_train, y_test)

### Scaling

In [ ]:
scaler = MinMaxScaler().fit(X_train)
X_trans = scaler.transform(X_train)
X_train_transformed = pd.DataFrame(X_trans, columns=X_train.columns, index=X_train.index)

In [ ]:
scaler = MinMaxScaler().fit(X_test)
X_testy = scaler.transform(X_test)
X_test_transformed = pd.DataFrame(X_testy, columns=X_test.columns, index=X_test.index)

### SMOTE

In [ ]:
smote = SMOTE()
X_smote_train, y_smote_train = smote.fit_sample(X_train_transformed, y_train) 
X_smote_test, y_smote_test = smote.fit_sample(X_test_transformed, y_test) 
smote_log = LogisticRegression(fit_intercept = False, C = 1e12,solver ='lbfgs', random_state=123)
smote_logit = smote_log.fit(X_smote_train, y_smote_train) 

In [ ]:
prediction_function(smote_logit, X_smote_train, X_smote_test, y_smote_train, y_smote_test)

In [ ]:
y_hats = model.predict(X)

df['y_hats'] = y_hats.reset_index()['name of the target column']

### Lasso Model

In [ ]:
log_lasso = LogisticRegression(penalty='l1', random_state=123)
logit_lasso = log_lasso.fit(X_train_transformed,y_train)
prediction_function(logit_lasso,X_train_transformed,X_test_transformed,y_train,y_test)

### Ridge Model (Balanced)

In [ ]:
ridge_logreg = LogisticRegression(class_weight='balanced', random_state=123)
ridge_logit = ridge_logreg.fit(X_train_transformed,y_train)

prediction_function(ridge_logit,X_train_transformed,X_test_transformed,y_train,y_test)

### Recursive Feature Elimination (RFE)

In [ ]:
logit = LogisticRegression()
rfe = RFE(logit, 10)
X_rfe_train = rfe.fit_transform(X_train_transformed,y_train)
X_rfe_test = rfe.transform(X_test_transformed)
rfe = rfe.fit(X_rfe_train,y_train.values.ravel())
prediction_function(rfe,X_rfe_train,X_rfe_test,y_train,y_test, 'rfe')

## Advanced Models

### Principal Component Analysis (PCA)

In [ ]:
pca_X2 = df_features
pca_y2 = df_target

pca_X_train2, pca_X_test2, pca_y_train2, pca_y_test2 = train_test_split(pca_X2, pca_y2, test_size=0.20, random_state=123)


In [ ]:
pca = PCA()
pca_train = pca.fit_transform(pca_X_train_transformed2)
pca_test = pca.transform(pca_X_test_transformed2)

In [ ]:
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
pca = PCA(n_components=15)
pca_X_train_transformed2 = pca.fit_transform(pca_X_train_transformed2)
pca_X_test_transformed2 = pca.transform(pca_X_test_transformed2)

In [ ]:
pca_df = pd.DataFrame({'var': pca.explained_variance_ratio_, 'Principal Component': ['PC1', 'PC2', 'PC3', 'PC4','PC5','PC6','PC7','PC8','PC9','PC10',
                                                                   'PC11', 'PC12', 'PC13', 'PC14','PC15']})
#                                                                     ,'PC16','PC17','PC18','PC19','PC20',
#                                                                    'PC21', 'PC22', 'PC23', 'PC24','PC25','PC26','PC27','PC28','PC29','PC30',
#                                                                    'PC31', 'PC32', 'PC33', 'PC34','PC35','PC36','PC37','PC38','PC39','PC40']})
plt.figure(figsize=(15,8))
sns.barplot(x='Principal Component', y='var', data=pca_df, color='c');

In [ ]:
print (pca.explained_variance_ratio_)

### Decision Tree

In [ ]:
X2 = df_features
y2 = df_target

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.20, random_state=123)

In [ ]:
mm_scaler = MinMaxScaler().fit(X_train2)
X_trans2 = mm_scaler.transform(X_train2)
X_train_transformed2 = pd.DataFrame(X_trans2, columns=X_train2.columns, index=X_train2.index)

mm_scaler = MinMaxScaler().fit(X_test2)
X_testy2 = mm_scaler.transform(X_test2)
X_test_transformed2 = pd.DataFrame(X_testy2, columns=X_test2.columns, index=X_test2.index)

In [ ]:
clf_tree = DecisionTreeClassifier(criterion='gini', max_depth=7, random_state=123, class_weight='balanced')
dec_tree = clf_tree.fit(X_train_transformed2, y_train2)

In [ ]:
y_pred = clf_tree.predict(X_test_transformed2)
max_depths = np.linspace(1, 32, 32, endpoint=True)
train_results = []
test_results = []
for max_depth in max_depths:
   dt = DecisionTreeClassifier(criterion='entropy', max_depth=max_depth)
   dt.fit(X_train_transformed2, y_train2)
   train_pred = dt.predict(X_train_transformed2)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_train2, train_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous train results
   train_results.append(roc_auc)
   y_pred = dt.predict(X_test_transformed2)
   false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test2, y_pred)
   roc_auc = auc(false_positive_rate, true_positive_rate)
   # Add auc score to previous test results
   test_results.append(roc_auc)
plt.figure(figsize=(12,6))
plt.plot(max_depths, train_results, 'b', label='Train AUC')
plt.plot(max_depths, test_results, 'r', label='Test AUC')
plt.ylabel('AUC score')
plt.xlabel('Tree depth')
plt.legend()
plt.show()

In [ ]:
prediction_function(dec_tree,X_train_transformed2,X_test_transformed2,y_train2,y_test2,estimate='dec_tree')

### Random Forest

In [ ]:
baseline_forest = RandomForestClassifier()
mean_baseline_forest = np.mean(cross_val_score(baseline_forest, X_train_transformed2, y_train2, cv=3))

print(f"Mean Cross Val Score for Random Forest: {mean_baseline_forest :.2%}")

In [ ]:
rf_param_grid = {
    'n_estimators': [10, 30, 100],
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 2, 6, 10],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [3, 6]
}

In [ ]:
rf_grid_search = GridSearchCV(baseline_forest, rf_param_grid, cv=3)
rf_grid_search.fit(X_train_transformed2, y_train2)

print(f"Training Accuracy: {rf_grid_search.best_score_ :.2%}")
print("")
print(f"Optimal Parameters: {rf_grid_search.best_params_}")

In [ ]:
forest = RandomForestClassifier(criterion='entropy', max_depth=6, min_samples_leaf=3, min_samples_split=5, n_estimators=10)
forest.fit(X_train_transformed2, y_train2)

In [ ]:
forest.score(X_train_transformed2, y_train2)

In [ ]:
forest.score(X_test_transformed2, y_test2)

In [ ]:
# random forest feature importance:
n=10
col_sorted_by_importance=forest.feature_importances_.argsort()[::-1][:n]
feat_imp=pd.DataFrame({
    'Features':X_train_transformed2.columns[col_sorted_by_importance],
    'Importance':forest.feature_importances_[col_sorted_by_importance]
})

import plotly.express as px
px.bar(feat_imp, x='Features', y='Importance',title="Random Forest Feature Importance")

### XGBoost

In [ ]:
xgb_clf_baseline = xgb.XGBClassifier()
xgb_clf_baseline.fit(X_train_transformed2, y_train2)
training_preds = xgb_clf_baseline.predict(X_train_transformed2)
val_preds = xgb_clf_baseline.predict(X_test_transformed2)
training_accuracy = accuracy_score(y_train2, training_preds)
val_accuracy = accuracy_score(y_test2, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

In [ ]:
param_grid = {
    "learning_rate": [1.0, 0.5, 0.1, 0.01],
    'max_depth': [1, 2, 4, 6, 10],
    'min_child_weight': [10],
    'subsample': [0.7, 0.8],
    'n_estimators': [3, 5, 30, 100, 250],
}

In [ ]:
grid_clf = GridSearchCV(xgb_clf_baseline, param_grid, scoring='accuracy', cv=3, n_jobs=4)
grid_clf.fit(X_train_transformed2, y_train2)

best_parameters = grid_clf.best_params_

print("Grid Search found the following optimal parameters: ")
for param_name in sorted(best_parameters.keys()):
    print("%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
xgb_clf = xgb.XGBClassifier(learning_rate=1.0, max_depth=2, min_child_weight=10, n_estimators=5, subsample=0.7)
xgb_clf.fit(X_train_transformed2, y_train2)
training_preds = xgb_clf.predict(X_train_transformed2)
val_preds = xgb_clf.predict(X_test_transformed2)
training_accuracy = accuracy_score(y_train2, training_preds)
val_accuracy = accuracy_score(y_test2, val_preds)

print("Training Accuracy: {:.4}%".format(training_accuracy * 100))
print("Validation accuracy: {:.4}%".format(val_accuracy * 100))

In [ ]:
# top 10 xgbost features:
xgb.plot_importance(xgb_clf_baseline,max_num_features=10)
pyplot.show()

In [ ]:
# xgboost feature importance:
n=10
col_sorted_by_importance=xgb_clf_baseline.feature_importances_.argsort()[::-1][:n]
feat_imp=pd.DataFrame({
    'Features':X_train_transformed2.columns[col_sorted_by_importance],
    'Importance':xgb_clf_baseline.feature_importances_[col_sorted_by_importance]
})

import plotly.express as px
px.bar(feat_imp, x='Features', y='Importance',title="XGBoost Feature Importance")

### AdaBoost Classifier

In [ ]:
# Instantiate an AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(random_state=123)
adaboost_clf.fit(X_train_transformed2, y_train2)
adaboost_train_preds = adaboost_clf.predict(X_train_transformed2)
adaboost_test_preds = adaboost_clf.predict(X_test_transformed2)

In [ ]:
adaboost_confusion_matrix = confusion_matrix(y_test2, adaboost_test_preds)
adaboost_confusion_matrix

In [ ]:
print("AdaBoost Training Metrics")
display_acc_and_f1_score(y_train2, adaboost_train_preds, model_name='AdaBoost')
print("")
print("AdaBoost Testing Metrics")
display_acc_and_f1_score(y_test2, adaboost_test_preds, model_name='AdaBoost')
print("")

In [ ]:
adaboost_classification_report = classification_report(y_test2, adaboost_test_preds)
print(adaboost_classification_report)

In [ ]:
print('Mean Adaboost Cross-Val Score (k=5):')
print(cross_val_score(adaboost_clf, X2, y2, cv=5).mean())

In [ ]:
adaboost_param_grid = {
    'n_estimators': [50, 100, 250],
    'learning_rate': [1.0, 0.5, 0.1]
}

In [ ]:
adaboost_grid_search = GridSearchCV(adaboost_clf, adaboost_param_grid, cv=3)
adaboost_grid_search.fit(X_train_transformed2, y_train2)

print(f"Training Accuracy: {adaboost_grid_search.best_score_ :.2%}")
print("")
print(f"Optimal Parameters: {adaboost_grid_search.best_params_}")

In [ ]:
adb_score = adaboost_grid_search.score(X_test_transformed2, y_test2)
print('Adaboost grid search: ', adb_score)

In [ ]:
# adaboost feature importance:
n=10
col_sorted_by_importance=adaboost_clf.feature_importances_.argsort()[::-1][:n]
feat_imp=pd.DataFrame({
    'Features':X_train_transformed2.columns[col_sorted_by_importance],
    'Importance':adaboost_clf.feature_importances_[col_sorted_by_importance]
})

import plotly.express as px
px.bar(feat_imp, x='Features', y='Importance',title="AdaBoost Feature Importance")

<a id="5.11"></a>

### Gradient Boosting Classifier

In [ ]:
# Instantiate a GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(random_state=123)
gbt_clf.fit(X_train_transformed2, y_train2)
gbt_clf_train_preds = gbt_clf.predict(X_train_transformed2)
gbt_clf_test_preds = gbt_clf.predict(X_test_transformed2)

In [ ]:
gbt_confusion_matrix = confusion_matrix(y_test2, gbt_clf_test_preds)
gbt_confusion_matrix

In [ ]:
print("GBT Training Metrics")
display_acc_and_f1_score(y_train2, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("GBT Testing Metrics")
display_acc_and_f1_score(y_test2, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

In [ ]:
gbt_classification_report = classification_report(y_test2, gbt_clf_test_preds)
print(gbt_classification_report)

In [ ]:
print('Mean GBT Cross-Val Score (k=5):')
print(cross_val_score(gbt_clf, X2, y2, cv=5).mean())

In [ ]:
# gradient boost feature importance:

n=10
col_sorted_by_importance=gbt_clf.feature_importances_.argsort()[::-1][:n]
feat_imp=pd.DataFrame({
    'Features':X_train_transformed2.columns[col_sorted_by_importance],
    'Importance':gbt_clf.feature_importances_[col_sorted_by_importance]
})

import plotly.express as px
px.bar(feat_imp, x='Features', y='Importance',title="Gradient Boost Feature Importance")

<a id="5.12"></a>

### K-Nearest Neighbors

In [ ]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(X_train_transformed2, y_train2)
knn_clf_train_preds = knn_clf.predict(X_train_transformed2)
knn_clf_test_preds = knn_clf.predict(X_test_transformed2)

In [ ]:
knn_confusion_matrix = confusion_matrix(y_test2, knn_clf_test_preds)
knn_confusion_matrix

In [ ]:
print("KNN Training Metrics")
display_acc_and_f1_score(y_train2, knn_clf_train_preds, model_name='K-Nearest Neighbors')
print("")
print("KNN Testing Metrics")
display_acc_and_f1_score(y_test2, knn_clf_test_preds, model_name='K-Nearest Neighbors')

In [ ]:
knn_classification_report = classification_report(y_test2, knn_clf_test_preds)
print(knn_classification_report)

In [ ]:
print('Mean KNN Cross-Val Score (k=5):')
print(cross_val_score(knn_clf, X2, y2, cv=5).mean())

<a id="5.13"></a>

### Ensemble / Meta-Classifier

In [ ]:
eclf = VotingClassifier(estimators=[('KNN', knn_clf), ('GBT', gbt_clf), ('XGB', xgb_clf), ('RF', forest)], voting='soft')
eclf.fit(X_train_transformed2, y_train2)
eclf_train_preds = eclf.predict(X_train_transformed2)
eclf_test_preds = eclf.predict(X_test_transformed2)

In [ ]:
print("ECLF Training Metrics")
display_acc_and_f1_score(y_train2, eclf_train_preds, model_name='ECLF')
print("")
print("ECLF Testing Metrics")
display_acc_and_f1_score(y_test2, eclf_test_preds, model_name='ECLF')
print("")

In [ ]:
eclf_classification_report = classification_report(y_test2, eclf_test_preds)
print(eclf_classification_report)

### Model Comparison - Intermediate


In [ ]:
model1 = model_score(baseline_log, X_train, X_test, y_train, y_test,"Baseline")
model2 = model_score(smote_logit, X_smote_train, X_smote_test, y_smote_train, y_smote_test,"SMOTE")
model3 = model_score(rfe,X_rfe_train,X_rfe_test,y_train,y_test,"RFE")
model4 = model_score(logit_lasso,X_train_transformed,X_test_transformed,y_train,y_test,"Lasso")
model5 = model_score(ridge_logit,X_train_transformed,X_test_transformed,y_train,y_test, "Ridge")

model_performances = pd.concat([model1,model2,model3,model4,model5],axis = 0).reset_index()
model_performances = model_performances.drop(columns = "index",axis =1)

dataframe  = ff.create_table(np.round(model_performances,4))

py.iplot(dataframe)

In [ ]:
model_performances

layout = go.Layout(dict(title = "Model Comparison",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "Metric",
                                     zerolinewidth=1,
                                     ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        margin = dict(l = 250),
                        height = 780))

trace1  = model_compare("Accuracy_score","#1bde65")
trace2  = model_compare('Recall_score',"#d93311")
trace3  = model_compare('Precision',"#3ae8d7")
trace4  = model_compare('f1_score',"#0f3175")

data = [trace1,trace2,trace3,trace4]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

<a id="6.2"></a>

### Model Comparison - Advanced

In [ ]:
def ML_model_score(model,train_x,test_x,train_y,test_y,name):
    model.fit(train_x,train_y)
    predictions  = model.predict(test_x)
    accuracy     = accuracy_score(test_y,predictions)
    recallscore  = recall_score(test_y,predictions)
    precision    = precision_score(test_y,predictions)
    roc_auc      = roc_auc_score(test_y,predictions)
    f1score      = f1_score(test_y,predictions) 
    cross_val    = cross_val_score(model,X2, y2, cv=5).mean()
    df_ML = pd.DataFrame({"Model"           : [name],
                       "Cross_Val"       : [cross_val],
                       "Accuracy_score"  : [accuracy],
                       "Recall_score"    : [recallscore],
                       "Precision"       : [precision],
                       "f1_score"        : [f1score],
                       "Area_under_curve": [roc_auc]})
    return df_ML

In [ ]:
ML_model1 = ML_model_score(dec_tree, X_train_transformed2, X_test_transformed2, y_train2, y_test2,"Decision Tree")
ML_model2 = ML_model_score(forest, X_train_transformed2, X_test_transformed2, y_train2, y_test2,"Random Forest")
ML_model3 = ML_model_score(xgb_clf,X_train_transformed2, X_test_transformed2, y_train2, y_test2,"XGBoost")
ML_model4 = ML_model_score(adaboost_clf,X_train_transformed2, X_test_transformed2, y_train2, y_test2,"AdaBoost")
ML_model5 = ML_model_score(gbt_clf,X_train_transformed2, X_test_transformed2, y_train2, y_test2, "GBT")
ML_model6 = ML_model_score(knn_clf,X_train_transformed2, X_test_transformed2, y_train2, y_test2, "KNN")

ML_model_performances = pd.concat([ML_model1,ML_model2,ML_model3,ML_model4,ML_model5,ML_model6],axis = 0).reset_index()
ML_model_performances = ML_model_performances.drop(columns = "index",axis =1)

ML_dataframe  = ff.create_table(np.round(ML_model_performances,4))

py.iplot(ML_dataframe)

In [ ]:
def ML_model_compare(metric,color) :
    ML_chart = go.Bar(y = ML_model_performances["Model"] ,
                    x = ML_model_performances[metric],
                    orientation = "h",name = metric ,
                    marker = dict(line = dict(width =.7),
                                  color = color))
    return ML_chart

In [ ]:
ML_model_performances

layout = go.Layout(dict(title = "ML Model Comparison",
                        plot_bgcolor  = "rgb(243,243,243)",
                        paper_bgcolor = "rgb(243,243,243)",
                        xaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     title = "Metric",
                                     zerolinewidth=1,
                                     ticklen=5,gridwidth=2),
                        yaxis = dict(gridcolor = 'rgb(255, 255, 255)',
                                     zerolinewidth=1,ticklen=5,gridwidth=2),
                        margin = dict(l = 250),
                        height = 780))

trace1  = ML_model_compare("Accuracy_score","#1bde65")
trace2  = ML_model_compare('Recall_score',"#d93311")
trace3  = ML_model_compare('Precision',"#3ae8d7")
trace4  = ML_model_compare('f1_score',"#0f3175")
trace5  = ML_model_compare('Cross_Val', "#ddf50a")
data = [trace1,trace2,trace3,trace4,trace5]
fig = go.Figure(data=data,layout=layout)
py.iplot(fig)

In [ ]:
# logistic regression predictions:

df_pred = df_copy.copy()
# create column for testing:
df_pred['Legalized']=df_pred['LegalStatus']

# update test column:
df_pred.iloc[:,-1].replace('Fully Legal','Yes',inplace=True)
df_pred.iloc[:,-1].replace('Fully Illegal','No',inplace=True)
df_pred.iloc[:,-1].replace('Mixed','No',inplace=True)

lrp = logreg.predict(X)
smt = smote_log.predict(X) #
las = log_lasso.predict(X) #
rid = ridge_logreg.predict(X) #
# rfelim = rfe.predict(X) #
# pca1 = pca.predict(X)
dt = dec_tree.predict(X) #
rf = forest.predict(X) #
xgb = xgb_clf.predict(X) #
adb = adaboost_clf.predict(X) #
gbc = gbt_clf.predict(X) #
knn = knn_clf.predict(X) #
ens = eclf.predict(X)

df_pred['LogRegPred'] = lrp
df_pred['SMOTEPred'] = smt
df_pred['LassoPred'] = las
df_pred['RidgePred'] = rid
# df_pred['RFEPred'] = rfelim
# df_pred['PCAPred'] = pca1
df_pred['DecisionTreePred'] = dt
df_pred['RandomForestPred'] = rf
df_pred['XGBoostPred'] = xgb
df_pred['AdaBoostPred'] = adb
df_pred['GradientBoostPred'] = gbc
df_pred['KNNPred'] = knn
df_pred['EnsemblePred'] = ens


df.columns[-2]
legal = ['Alaska','California','Colorado','Maine','Massachusetts','Nevada',
'Oregon','Washington','Illinois','Michigan','Vermont']

In [ ]:
# find out states from model predictions:
z=[]
for x in df_pred.columns[-11:]:
    z.append((np.unique(df_pred['State_Code'].loc[(df_pred[x] == 1) & (df_pred['Legalized'] =='No')].unique())))

# flatten nested list:
flatList = []
for elem in z:
    for item in elem:
        flatList.append(item)
# print('Flat List : ', flatList)

# get model counts for each state and sort in descending order:
counts,values = pd.Series(flatList).value_counts().values, pd.Series(flatList).value_counts().index
df_results = pd.DataFrame(list(zip(values,counts)),columns=["value","count"])
df_results

In [ ]:
np.unique(df_pred['State_Code'].loc[(df_pred['KNNPred'] == 1) & (df_pred['Legalized'] =='No')].unique())

In [ ]:
np.unique(df_pred['State_Code'].loc[(df_pred['GradientBoostPred'] == 1) & (df_pred['Legalized'] =='No')].unique())

In [ ]:
# counts,values = df_pred.values(), df_pred.index
# df_results = pd.DataFrame(list(zip(values,counts)),columns=["value","count"])

fig = px.bar(df_results, x='value', y='count')
fig.show()

In [ ]:
df_pred.head()

In [ ]:
feat = [
'Employment Industry',
'Religion',
'Ancestry',
'Ancestry',
'Commute-Walk',
'Ancestry',
'Ancestry',
'Home Heating Fuel - Other',
'Ancestry',
'Ancestry',
'Ancestry',
'Home Heating Fuel - Wood',
'Ancestry',
'Ancestry',
'Ancestry',
'Ancestry',
'Ancestry',
'Fertility',
'Medicinal',
'Ancestry',
'Home Heating Fuel - Wood',
'Ancestry',
'Ancestry',
'Fertility',
'Ancestry',
'Religion',
'Commute-Walk',
'Race',
'Sex and Age Median',
'Race',
'Sex and Age per 100 females - 65 and over',
'Race',
'Sex and Age per 100 females - 18 and over',
'Race',
'Race',
'Race',
'Party Affiliation',
'Home Heating Fuel - Oil',
'Race',
'Ancestry',
'Housing Costs',
'Race',
'House Tenure',
'Ancestry',
'Ancestry',
'Medicinal',
'Party Affiliation',
'Ancestry',
'Home Value',
'Ancestry',
'Party Affiliation',
'Home Heating Fuel - Other',
'Relgion',
'Ancestry',
'Party Affiliation',
'Medicinal',
'Home Heating Fuel - Electricity',
'Home Heating Fuel - Wood',
'Home Structure',
'Population',
'Number of Rooms in Home',
'Home Heating Fuel - Oil',
'Employment Industry',
'Religion',
'Home Heating Fuel - Oil',
'Ancestry',
'Party Affiliation',
'Home Vacancy Rate',
'Housing Costs',
'Population',
'Employment Industry',
'Housing Costs',
'Party Affiliation',
'Home Heating Fuel - Oil',
'Ancestry',
'Home Heating Fuel - Wood',
'Home Heating Fuel - Electricity',
'Number of Rooms in Home',
'Medicinal',
'Employment Industry',
'Ancestry'
]

In [ ]:
counts,values = pd.Series(feat).value_counts().values, pd.Series(feat).value_counts().index
df_feat = pd.DataFrame(list(zip(values,counts)),columns=["value","count"])
df_feat

In [ ]:
fig = px.bar(df_feat, x='value', y='count')
fig.show()